# Prism V2 Demo - K8

In [42]:
%autoawait
import time
import re
import qrcode
import uuid
import base64
import jwt
# https://pypi.org/project/termcolor/
from termcolor import colored,cprint
import os

from dotenv import load_dotenv
load_dotenv()
APIKEY = os.getenv('APIKEY')

IPython autoawait is `on`, and set to use `asyncio`


## Import Prism V2 Controller Library

In [43]:
from prism_agent_open_api_specification_client import AuthenticatedClient

## Instantiate K8 Authenticated Agent Controllers

In [44]:
from prism_agent_open_api_specification_client import AuthenticatedClient

# {
#   id     = "agent-sd67m"
#   domain = "agent-sd67m.atalaprism.io"
# },
# {
#   id     = "agent-gr27n"
#   domain = "agent-gr27n.atalaprism.io"
# },
# {
#   id     = "agent-er56m"
#   domain = "agent-er56m.atalaprism.io"
# }

issuer_client_controller = AuthenticatedClient(base_url="https://agent-sd67m.atalaprism.io/prism-agent", token=APIKEY)
holder_client_controller = AuthenticatedClient(base_url="https://agent-gr27n.atalaprism.io/prism-agent", token=APIKEY)
verifier_client_controller = AuthenticatedClient(base_url="https://agent-er56m.atalaprism.io/prism-agent", token=APIKEY)

## Import Connection Management Models and API Client

In [45]:
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.types import Response

## Check Agent Connections

In [46]:
issuer_conns: Response[ConnectionCollection] = get_connections.sync(client=issuer_client_controller)
holder_conns: Response[ConnectionCollection] = get_connections.sync(client=holder_client_controller)
verifier_conns: Response[ConnectionCollection] = get_connections.sync(client=verifier_client_controller)
print("Issuer Connections:\n{}\nHolder Connections:\n{}\nVerifier Connections:\n{}".format(issuer_conns, holder_conns,verifier_conns))

Issuer Connections:
ConnectionCollection(self_='/collections', kind='Collection', contents=[Connection(self_='Connection', kind='/connections/ffb61ca1-be69-4ba8-9639-a36fea06bd63', connection_id='ffb61ca1-be69-4ba8-9639-a36fea06bd63', state=<ConnectionAllOfState.ConnectionResponseSent: 'ConnectionResponseSent'>, created_at=datetime.datetime(2022, 12, 19, 23, 45, 23, tzinfo=tzutc()), invitation=ConnectionInvitation(id='ffb61ca1-be69-4ba8-9639-a36fea06bd63', type='https://didcomm.org/out-of-band/2.0/invitation', from_='did:peer:2.Ez6LSi1DoS8BjWLGRJvT2iwTv9UvHJASC6p7CdRtvPwBUUzQa.Vz6Mkty3d3dfgEVzWUQfZpnaXgC4YySrYpk1SHoJDqYtqChZV.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9hZ2VudC1zZDY3bS5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', invitation_url='https://domain.com/path?_oob=eyJpZCI6ImZmYjYxY2ExLWJlNjktNGJhOC05NjM5LWEzNmZlYTA2YmQ2MyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNpMURvUzhCaldMR1JKdlQya

## Issuer Connects with Holder

In [23]:
# Create Invitation
conn_request = CreateConnectionRequest()
conn_request.label = 'Connect with Holder'
issuer_invitation: Response[Connection] =  create_connection.sync(client=issuer_client_controller,json_body=conn_request)

print("Response:\n",issuer_invitation)
print("Connection ID\n",issuer_invitation.connection_id)
# print("Invitation URL\n",issuer_invitation.invitation.invitation_url)
issuer_invitationUrlbase64 = re.sub("^.*_oob=", "", issuer_invitation.invitation.invitation_url)
print("Invitation Base64\n",issuer_invitationUrlbase64)

Response:
 Connection(self_='Connection', kind='/connections/ffb61ca1-be69-4ba8-9639-a36fea06bd63', connection_id='ffb61ca1-be69-4ba8-9639-a36fea06bd63', state=<ConnectionAllOfState.InvitationGenerated: 'InvitationGenerated'>, created_at=datetime.datetime(2022, 12, 19, 23, 45, 23, 285805, tzinfo=tzutc()), invitation=ConnectionInvitation(id='ffb61ca1-be69-4ba8-9639-a36fea06bd63', type='https://didcomm.org/out-of-band/2.0/invitation', from_='did:peer:2.Ez6LSi1DoS8BjWLGRJvT2iwTv9UvHJASC6p7CdRtvPwBUUzQa.Vz6Mkty3d3dfgEVzWUQfZpnaXgC4YySrYpk1SHoJDqYtqChZV.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9hZ2VudC1zZDY3bS5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', invitation_url='https://domain.com/path?_oob=eyJpZCI6ImZmYjYxY2ExLWJlNjktNGJhOC05NjM5LWEzNmZlYTA2YmQ2MyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNpMURvUzhCaldMR1JKdlQyaXdUdjlVdkhKQVNDNnA3Q2RSdHZQd0JVVXpRYS5WejZNa3R5M2QzZGZnRVZ6V1VRZlpwbmFYZ0M0WXlT

## Holder Accepts Invitation from Issuer

In [24]:
# Accept Invitation
accept_conn_request = AcceptConnectionInvitationRequest(issuer_invitationUrlbase64)
holder_invitation: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=holder_client_controller,json_body=accept_conn_request)
print(holder_invitation)

Connection(self_='Connection', kind='/connections/83e50a9e-a1fa-4e17-9b1c-a19da204ce3c', connection_id='83e50a9e-a1fa-4e17-9b1c-a19da204ce3c', state=<ConnectionAllOfState.ConnectionRequestPending: 'ConnectionRequestPending'>, created_at=datetime.datetime(2022, 12, 19, 23, 45, 25, tzinfo=tzutc()), invitation=ConnectionInvitation(id='ffb61ca1-be69-4ba8-9639-a36fea06bd63', type='https://didcomm.org/out-of-band/2.0/invitation', from_='did:peer:2.Ez6LSi1DoS8BjWLGRJvT2iwTv9UvHJASC6p7CdRtvPwBUUzQa.Vz6Mkty3d3dfgEVzWUQfZpnaXgC4YySrYpk1SHoJDqYtqChZV.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9hZ2VudC1zZDY3bS5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', invitation_url='https://domain.com/path?_oob=eyJpZCI6ImZmYjYxY2ExLWJlNjktNGJhOC05NjM5LWEzNmZlYTA2YmQ2MyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNpMURvUzhCaldMR1JKdlQyaXdUdjlVdkhKQVNDNnA3Q2RSdHZQd0JVVXpRYS5WejZNa3R5M2QzZGZnRVZ6V1VRZlpwbmFYZ0M0WXlTcllwazFTS

## Check Issuer and Holder Connection State

In [25]:
# Check Connection State
issuer_conn: Response[Connection] =  get_connection.sync(client=issuer_client_controller,connection_id=issuer_invitation.connection_id)
holder_conn: Response[Connection] =  get_connection.sync(client=holder_client_controller,connection_id=holder_invitation.connection_id)

while (issuer_conn.state != 'ConnectionResponseSent' and holder_conn.state != 'ConnectionResponseReceived'):
    issuer_conn: Response[Connection] =  get_connection.sync(client=issuer_client_controller,connection_id=issuer_invitation.connection_id)
    holder_conn: Response[Connection] =  get_connection.sync(client=holder_client_controller,connection_id=holder_invitation.connection_id)
    print("Issuer State: {} Holder State: {} \n".format(issuer_conn.state,holder_conn.state))
    time.sleep(1)
    
print("Connection established between Issuer and Holder!")

Issuer State: InvitationGenerated Holder State: ConnectionRequestPending 

Issuer State: InvitationGenerated Holder State: ConnectionRequestPending 

Issuer State: ConnectionResponsePending Holder State: ConnectionRequestPending 

Issuer State: ConnectionResponseSent Holder State: ConnectionRequestSent 

Connection established between Issuer and Holder!


## Check Agent Connections

In [26]:
issuer_conns: Response[ConnectionCollection] = get_connections.sync(client=issuer_client_controller)
holder_conns: Response[ConnectionCollection] = get_connections.sync(client=holder_client_controller)
verifier_conns: Response[ConnectionCollection] = get_connections.sync(client=verifier_client_controller)
print("Issuer Connections:\n{}\n\nHolder Connections:\n{}\n\nVerifier Connections:\n{}".format(issuer_conns, holder_conns,verifier_conns))

Issuer Connections:
ConnectionCollection(self_='/collections', kind='Collection', contents=[Connection(self_='Connection', kind='/connections/ffb61ca1-be69-4ba8-9639-a36fea06bd63', connection_id='ffb61ca1-be69-4ba8-9639-a36fea06bd63', state=<ConnectionAllOfState.ConnectionResponseSent: 'ConnectionResponseSent'>, created_at=datetime.datetime(2022, 12, 19, 23, 45, 23, tzinfo=tzutc()), invitation=ConnectionInvitation(id='ffb61ca1-be69-4ba8-9639-a36fea06bd63', type='https://didcomm.org/out-of-band/2.0/invitation', from_='did:peer:2.Ez6LSi1DoS8BjWLGRJvT2iwTv9UvHJASC6p7CdRtvPwBUUzQa.Vz6Mkty3d3dfgEVzWUQfZpnaXgC4YySrYpk1SHoJDqYtqChZV.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9hZ2VudC1zZDY3bS5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', invitation_url='https://domain.com/path?_oob=eyJpZCI6ImZmYjYxY2ExLWJlNjktNGJhOC05NjM5LWEzNmZlYTA2YmQ2MyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNpMURvUzhCaldMR1JKdlQya

## Import Credential Management Models and API Client

In [27]:
from prism_agent_open_api_specification_client.models import IssueCredentialRecord, CreateIssueCredentialRecordRequest, CreateIssueCredentialRecordRequestClaims, IssueCredentialRecordCollection
from prism_agent_open_api_specification_client.api.issue_credentials_protocol import get_credential_record, get_credential_records, create_credential_offer,accept_credential_offer,issue_credential
from prism_agent_open_api_specification_client.types import Response, Unset

## Check Credentials in Wallets

In [28]:
issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
verifier_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=verifier_client_controller)
print("Issuer Credentials:\n{}\n\nHolder Credentials:\n{}\n\nVerifier Credentials:\n{}".format(issuer_creds, holder_creds,verifier_creds))

Issuer Credentials:
IssueCredentialRecordCollection(items=[IssueCredentialRecord(subject_id='did:peer:2.Ez6LSejUdL5Xv6iREpd5x82psUfBAvgypqHfJ3t9S2WDnkLRr.Vz6MkqCAankSAS4Qi4oGyRt7GLGVGx9ZgSxLWoep3HFstcscK.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9hZ2VudC1ncjI3bi5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', claims=CreateIssueCredentialRecordRequestClaims(additional_properties={'birthdate': '01/01/2000', 'firstname': 'Alice', 'lastname': 'Wonderland'}), record_id='348f02c5-29a3-47ab-8487-6c2e472dcb6c', created_at=datetime.datetime(2022, 12, 19, 23, 3, 21, tzinfo=tzutc()), role=<IssueCredentialRecordAllOfRole.ISSUER: 'Issuer'>, protocol_state=<IssueCredentialRecordAllOfProtocolState.CREDENTIALSENT: 'CredentialSent'>, schema_id='1234', validity_period=3600.0, automatic_issuance=False, await_confirmation=False, updated_at=datetime.datetime(2022, 12, 19, 23, 4, 7, tzinfo=tzutc()), publication_state=<prism_agent_open_api_specification_client.types.Unset object at 0

## Define Credential Schema and Content

In [29]:
data = {
        "firstname": "Alice",
        "lastname": "Wonderland",
        "birthdate": "01/01/2000"
      }
cred_claims = CreateIssueCredentialRecordRequestClaims()
cred_claims = cred_claims.from_dict(data)
print(cred_claims)

credential_offer = CreateIssueCredentialRecordRequest(subject_id=issuer_conn.their_did, claims=cred_claims, schema_id="1234", validity_period=3600, automatic_issuance=False, await_confirmation=False)

print(credential_offer.to_dict())

CreateIssueCredentialRecordRequestClaims(additional_properties={'firstname': 'Alice', 'lastname': 'Wonderland', 'birthdate': '01/01/2000'})
{'subjectId': 'did:peer:2.Ez6LSsnhMrkzLMyr5o8RyWtRg44Ap62gieWz684tkcQMB7DLQ.Vz6Mkm4BRoKpCcPJTWRezAkM5fLtKmcUzaFqQ6fr5cGtbhgQZ.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9hZ2VudC1ncjI3bi5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', 'claims': {'firstname': 'Alice', 'lastname': 'Wonderland', 'birthdate': '01/01/2000'}, 'schemaId': '1234', 'validityPeriod': 3600, 'automaticIssuance': False, 'awaitConfirmation': False}


## Send Credential Offer to Holder

In [30]:
# Create Credential Offer
issuer_cred_offer: Response[IssueCredentialRecord] = create_credential_offer.sync(client=issuer_client_controller,json_body=credential_offer)
print(issuer_cred_offer)

IssueCredentialRecord(subject_id='did:peer:2.Ez6LSsnhMrkzLMyr5o8RyWtRg44Ap62gieWz684tkcQMB7DLQ.Vz6Mkm4BRoKpCcPJTWRezAkM5fLtKmcUzaFqQ6fr5cGtbhgQZ.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9hZ2VudC1ncjI3bi5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', claims=CreateIssueCredentialRecordRequestClaims(additional_properties={'birthdate': '01/01/2000', 'firstname': 'Alice', 'lastname': 'Wonderland'}), record_id='0728a12f-bf17-4631-b8fa-c282ad59a3b1', created_at=datetime.datetime(2022, 12, 19, 23, 45, 57, 807151, tzinfo=tzutc()), role=<IssueCredentialRecordAllOfRole.ISSUER: 'Issuer'>, protocol_state=<IssueCredentialRecordAllOfProtocolState.OFFERPENDING: 'OfferPending'>, schema_id='1234', validity_period=3600.0, automatic_issuance=False, await_confirmation=False, updated_at=<prism_agent_open_api_specification_client.types.Unset object at 0x7f127eccf940>, publication_state=<prism_agent_open_api_specification_client.types.Unset object at 0x7f127eccf940>, jwt_credential=

## Retrieve Credential Records from Issuer and Holder

In [36]:
# Check Credential State
issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
verifier_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=verifier_client_controller)

print(colored("Current state for Issuer Credential Record {} is {} and Holder Credential Record {} is {}".format(issuer_creds.items[0].record_id,issuer_creds.items[0].protocol_state,holder_creds.items[0].record_id,holder_creds.items[0].protocol_state), "magenta", attrs=["bold"]))


Current state for Issuer Credential Record 348f02c5-29a3-47ab-8487-6c2e472dcb6c is CredentialSent and Holder Credential Record 093ff453-9bfd-4255-91a5-746435ca8179 is CredentialReceived


## Holder Accept Credential Offer from Issuer

In [39]:
print(holder_creds.items[0].record_id)
holder_cred_record: Response[IssueCredentialRecord] = accept_credential_offer.sync(client=holder_client_controller, record_id=holder_creds.items[0].record_id)
print(holder_cred_record)

093ff453-9bfd-4255-91a5-746435ca8179
None


## Issue Credential to Holder

In [40]:
print(issuer_creds.items[0].record_id)
issuer_cred_record: Response[IssueCredentialRecord] = issue_credential.sync(client=issuer_client_controller, record_id=issuer_creds.items[0].record_id)
print(issuer_cred_record)

348f02c5-29a3-47ab-8487-6c2e472dcb6c
None


## Check if Credential is Successfully Issued to Holder

In [41]:
# Check Credential State
issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
verifier_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=verifier_client_controller)

print(colored("Current state for Issuer Credential Record {} is {} and Holder Credential Record {} is {}".format(issuer_creds.items[0].record_id,issuer_creds.items[0].protocol_state,holder_creds.items[0].record_id,holder_creds.items[0].protocol_state), "magenta", attrs=["bold"]))

while (issuer_creds.items[0].protocol_state != 'CredentialSent' and holder_creds.items[0].protocol_state != 'CredentialReceived'):
    issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
    holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
    print("Issuer State: {} Holder State: {} \n".format(issuer_creds.items[0].protocol_state,holder_creds.items[0].protocol_state))
    time.sleep(1)
    
print(colored("Credential Record {} Issued to Holder! Continue with notebook".format(issuer_creds.items[0].record_id), "green", attrs=["bold"]))

Current state for Issuer Credential Record 348f02c5-29a3-47ab-8487-6c2e472dcb6c is CredentialSent and Holder Credential Record 093ff453-9bfd-4255-91a5-746435ca8179 is CredentialReceived
Credential Record 348f02c5-29a3-47ab-8487-6c2e472dcb6c Issued to Holder! Continue with notebook


## DIDs

## Import DID Management Models and API Client

In [ ]:
from prism_agent_open_api_specification_client.models import DID, DIDDocumentMetadata, DIDOperationResponse, DIDResponse, CreateManagedDidRequest, ListManagedDIDResponseInner, CreateManagedDIDResponse
from prism_agent_open_api_specification_client.api.did import get_did
from prism_agent_open_api_specification_client.api.did_registrar import create_managed_did,list_managed_did,publish_managed_did
from prism_agent_open_api_specification_client.types import Response

### List DIDs in Wallet

In [ ]:
issuer_dids: Response[ListManagedDIDResponseInner] = list_managed_did.sync(client=issuer_client_controller)
print(issuer_dids)
print(issuer_dids[0].did)

### Create DID

In [ ]:
data = {
    "documentTemplate": {
      "publicKeys": [
        {
          "id": "auth0",
          "purpose": "authentication"
        }
      ],
      "services": []
    }
  }
new_did = CreateManagedDidRequest.from_dict(data)

issuer_new_did: Response[CreateManagedDIDResponse] = create_managed_did.sync(client=issuer_client_controller, json_body=new_did)
print(issuer_new_did)

### Resolve DID

In [ ]:
issuer_resolve_did: Response[DID] = get_did.sync(client=issuer_client_controller, did_ref=issuer_new_did.long_form_did)
print(issuer_resolve_did)

issuer_resolve_did: Response[DID] = get_did.sync(client=issuer_client_controller, did_ref=issuer_dids[0].did)
print(issuer_resolve_did)


### Publish DID

In [ ]:
issuer_publish_did: Response[DID] = publish_managed_did.sync(client=issuer_client_controller, did_ref=issuer_new_did.long_form_did)
print(issuer_publish_did)

### Deactivate DID